In [1]:
%load_ext autoreload

%autoreload 2

## Как было раньше

In [ ]:
import time
import os

pid = os.fork()
if pid == 0:
    # дочерний процесс
    while True:
        print("child:", os.getpid())
        time.sleep(5)
else:
    # родительский процесс
    print("parent:", os.getpid())
    os.wait()

In [9]:
os.system("dir")

0

#### Задача 1.

Задача: суммирование всех элементов массива

In [6]:
size = 10 * 1000 * 1000

Синхронный запуск

In [7]:
arr = [1 for _ in range(size)]

In [6]:
%%timeit
_ = sum(arr)

10 loops, best of 3: 108 ms per loop


#### Задача 2.

Файл reader/search_text.py: функция, которая делает запросы к вики

In [ ]:
def search_text(args, cons=False):
    tmp, query = args
    if cons:
        print(query)
    url = tmp.format(urlencode({"search": query}))
    req = get(url)
    assert req.status_code == 200, "request failed"
    soup = BeautifulSoup(req.text, "lxml")
    return " ".join(list(map(lambda x: str(x.text).replace("\n", " ").strip(),
                              soup.find("div", {'id': 'mw-content-text'}).findAll("p")[2:])))

In [1]:
# Файл reader/reader.py
from reader import WikiReader

In [2]:
wr = WikiReader(["Cat", "Alpaca", "Iguana", "Monkey", "Goat", "Lion", "Panda"])

Синхронный запуск поиска

In [ ]:
def run_synchronously(self):
    return dict(zip(self.__queries, map(self.search_text,
                                        zip([self.__tmp] * len(self.__queries), self.__queries))))

In [4]:
%%timeit
_ = wr.run_synchronously()

1 loop, best of 3: 4.9 s per loop


## subprocess

In [11]:
import subprocess as sp

### Popen

In [12]:
p = sp.Popen(["git", "help", "-a"])
p

#### Получение результатов подпроцесса

In [13]:
p.stdout

In [14]:
p = sp.Popen(["git", "help", "-a"], stdout=sp.PIPE)
p.wait()
list(p.stdout)[:3]

[b'usage: git [--version] [--help] [-C <path>] [-c name=value]\n',
 b'           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]\n',
 b'           [-p | --paginate | --no-pager] [--no-replace-objects] [--bare]\n']

### run

In [15]:
# Файл sum.cpp
"""
#include <iostream>

int Sum(int first, int second) {
    return first + second;
}

int main() {
    int first = 0;
    int second = 0;
    std::cin >> first >> second;
    std::cout << Sum(first, second) << std::endl;
}
"""

sp.run(['g++', 'sum.cpp', '-o', 'sum.out'], stderr=sp.PIPE)

CompletedProcess(args=['g++', 'sum.cpp', '-o', 'sum.out'], returncode=0, stderr=b'')

In [82]:
sp.run(['./sum.out'], stdout=sp.PIPE, input=b'2 3')

CompletedProcess(args=['./sum.out'], returncode=0, stdout=b'5\r\n')

### Handling errors

In [72]:
# Файл bad_sum.cpp
"""
// #include <iostream>

int Sum(int first, int second) {
    return first + second;
}

int main() {
    int first = 0;
    int second = 0;
    std::cin >> first >> second;
    std::cout << Sum(first, second) << std::endl;
}
"""

sp.run(['g++', 'bad_sum.cpp', '-o', 'sum.out'], stderr=sp.PIPE)

CompletedProcess(args=['g++', 'bad_sum.cpp', '-o', 'sum.out'], returncode=1, stderr=b"bad_sum.cpp: In function 'int main()':\r\nbad_sum.cpp:10:5: error: 'cin' is not a member of 'std'\r\n     std::cin >> a >> b;\r\n     ^\r\nbad_sum.cpp:11:5: error: 'cout' is not a member of 'std'\r\n     std::cout << Sum(a, b) << '\\n';\r\n     ^\r\n")

### shlex - строка аргументов

In [76]:
import shlex
sp.run(shlex.split('g++ sum.cpp -o sum.out'))

CompletedProcess(args=['g++', 'sum.cpp', '-o', 'sum.out'], returncode=0)

### Тестируем на вики

Асинхронный запуск поиска с помощью subprocess

In [ ]:
class WikiReader(object):
    ...
    def run_async_sub_proc(self):
        def communicate(sub_process):
            out, _ = sub_process.communicate()
            return out.decode("utf-8").split("\n")

        sub_processes = [
            sp.Popen(["python", "./reader/search_text.py", self.__tmp, query],
                     stdout=sp.PIPE, stderr=sp.PIPE)
            for query in self.__queries]
        res = []
        for sub_process, query in zip(sub_processes, self.__queries):
            res.append(communicate(sub_process))
        if not all(map(lambda x: x[0][0].strip() == x[1], zip(res, self.__queries))):
            print("Failed on results order")
        return dict(zip(self.__queries, list(map(lambda x: x[1], res))))
    ...

In [8]:
%%timeit
_ = wr.run_async_sub_proc()

1 loop, best of 3: 4.62 s per loop


## threading

In [1]:
import threading

In [2]:
def thread_job(number):
    # time.sleep(np.random.rand())
    print(" Hi! I am {} ".format(number), end = " | ")


def run_threads(count):
    threads = [
        threading.Thread(target=thread_job, args=(i,))
        for i in range(count)
    ]
    for thread in threads:
        thread.start()
    # ALL NON-DAEMON THREADS MUST BE JOINED
    for thread in threads:
        thread.join()

run_threads(4)

 Hi! I am 0  Hi! I am 1  Hi! I am 2  Hi! I am 3  |  |  |  | 

Далее примеры из https://www.youtube.com/watch?v=Bv25Dwe84g0

In [5]:
import random
import time

counter = 0
def thread_job():
    global counter
    old_counter = counter
    time.sleep(random.randint(0, 1))
    counter = old_counter + 1
    print('{} '.format(counter), end='')

threads = [threading.Thread(target=thread_job) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()
counter

1 2 3 4 5 2 5 5 6 5 

5

### Блокировки

In [16]:
counter = 0
def thread_job(lock):
    time.sleep(random.randint(0, 1))
    lock.acquire()
    global counter
    counter += 1
    print('{} '.format(counter), end='')
    lock.release()

lock = threading.Lock()
threads = [
    threading.Thread(target=thread_job, args=(lock,))
    for i in range(10)
]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

counter

1 2 3 4 5 6 7 9 8 10 

10

In [6]:
counter = 0
def thread_job(lock):
    with lock:
        time.sleep(random.randint(0, 1))
        global counter
        counter += 1
        print('{} '.format(counter), end='')

lock = threading.Lock()
threads = [
    threading.Thread(target=thread_job, args=(lock,))
    for i in range(10)
]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

counter

1 2 3 4 5 6 7 8 9 10 

10

#### Если вдруг используете блокировки, то только RLock

http://asvetlov.blogspot.ru/2010/11/2.html

In [ ]:
import threading


class Point(object):
    def __init__(self):
        self._mutex = threading.RLock()
        self._x = 0
        self._y = 0

    def get(self):
        with self._mutex:
            return (self._x, self._y)

    def set(self, x, y):
        with self._mutex:
            self._x = x
            self._x = y

In [ ]:
class ColoredPoint(Point):
    def __init__(self):
        super(ColoredPoint, self).__init__()
        self._color = 'green'

    @property
    def color(self):
        with self._mutex:
            return self._color

    @color.setter
    def color(self, val):
        with self._mutex:
            self._color = val

    def do(self, observer):
        with self._mutex:
            if self._color == 'red':
                observer(self.get())

### Правильное решение (очереди)

In [20]:
from queue import Queue
import threading
class Counter(object):
    def __init__(self, value=0):
        self.value = value
def thread_job_adder(counter, input_q, printer_q):
    while True:
        counter.value += input_q.get()
        input_q.task_done()
        printer_q.put(counter.value)
def thread_job_printer(printer_q):
    while True:
        print(printer_q.get(), end=' ')
        printer_q.task_done()
def data_generator(input_q):
    input_q.put(1)
def get_counted(size=10):
    counter = Counter()
    input_q = Queue()
    printer_q = Queue()
    adder_daemon = threading.Thread(target=thread_job_adder,
                                         args=(counter, input_q, printer_q), daemon=True)
    adder_daemon.start()
    
    printer_daemon = threading.Thread(target=thread_job_printer,
                                         args=(printer_q, ), daemon=True)
    printer_daemon.start()
    
    threads = [threading.Thread(target=data_generator, args=(input_q, ))
                   for query in range(size)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    input_q.join()
    printer_q.join()


In [21]:
get_counted()

1 2 3 4 5 6 7 8 9 10 

### Тестируем на сумму

In [1]:
size = 10 * 1000 * 1000
arr = [1 for _ in range(size)]
process_count = 8
part_size = size // process_count
data = [arr[i * part_size: (i + 1) * part_size] for i in range(process_count)]

In [10]:
import queue

def thread_job(arr, results_queue):
    results_queue.put(sum(arr))

def sum_using_threads(data):
    results_queue = queue.Queue()
    threads = [
        threading.Thread(target=thread_job, args=(batch, results_queue))
        for batch in data]
    for thread in threads:
        thread.start()

    results = []
    for thread in threads:
        results.append(results_queue.get())
        thread.join()

    return sum(results)

In [11]:
%%timeit
sum(arr)

10 loops, best of 3: 110 ms per loop


In [12]:
%%timeit
sum_using_threads(data)

10 loops, best of 3: 114 ms per loop


### Тестируем на вики

Асинхронный запуск поиска с помощью потоков

In [ ]:
class WikiReader(object):
    ...
    def run_async_thread(self):
        res = list()

        def thread_job(tmp, query):
            res.append((query, self.search_text((tmp, query))))

        threads = [threading.Thread(target=thread_job, args=(self.__tmp, query,))
                   for query in self.__queries]

        for thread in threads:
            thread.start()

        for thread in threads:
            thread.join()

        if not all(map(lambda x: x[0][0].strip() == x[1], zip(res, self.__queries))):
            print("Failed on results order")

        return dict(zip(self.__queries, list(map(lambda x: x[1], res))))
    ...

In [7]:
%%timeit
_ = wr.run_async_thread()

Failed on results order
Failed on results order
Failed on results order
Failed on results order
1 loop, best of 3: 2.69 s per loop


Асинхронный запуск поиска с помощью потоков (выстраиваем порядок) - неправильная реализация (Почему?)

In [ ]:
class WikiReader(object):
    ...
    def run_async_thread_with_q(self):
        def thread_job(input_q, output_q):
            while True:
                tmp, query = input_q.get()
                res = (query, self.search_text((tmp, query)))
                output_q.put(res)
                input_q.task_done()

        def data_generator(input_q, tmp, query):
            input_q.put((tmp, query))

        input_q = Queue()
        output_q = Queue()

        thread_job_daemon = threading.Thread(target=thread_job,
                             args=(input_q, output_q), daemon=True)
        thread_job_daemon.start()

        threads = [threading.Thread(target=data_generator, args=(input_q, self.__tmp, query, ))
                   for query in self.__queries]
        for thread in threads:
            thread.start()
        res = []
        for thread, query in zip(threads, self.__queries):
            res.append(output_q.get())
            thread.join()
        if not all(map(lambda x: x[0][0].strip() == x[1], zip(res, self.__queries))):
            print("Failed on results order")
        input_q.join()
        return dict(zip(self.__queries, list(map(lambda x: x[1], res))))
    ...

In [12]:
%%timeit
res = wr.run_async_thread_with_q()

1 loop, best of 3: 4.95 s per loop


### ThreadPoolExecutor

In [ ]:
# since Python 3.2
from concurrent.futures import ThreadPoolExecutor
%%timeit
executor = ThreadPoolExecutor(max_workers=8)
list(executor.map(sum_using_threads, data))

In [3]:
%%timeit
from concurrent.futures import ThreadPoolExecutor

executor = ThreadPoolExecutor(max_workers=8)
futures = []
for batch in data:
    future_result = executor.submit(sum, batch)
    futures.append(future_result)
results = [f.result() for f in futures]

10 loops, best of 3: 116 ms per loop


## multiprocessing

In [18]:
from multiprocessing import Process

def f(name):
    print('hello', name)


p = Process(target=f, args=('bob',))
p.start()
p.join()

In [ ]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])
    print("task_done()")
    q.task_done()

q = Queue()
p = Process(target=f, args=(q,))
print("p")
p.start()
print("q get")
print(q.get())
print("before")
q.join()
p.join()

### Тестируем на задачи суммирования

In [22]:
import multiprocessing

In [23]:
size = 10 * 1000 * 1000
arr = [1 for _ in range(size)]
process_count = 8
part_size = size // process_count
data = [arr[i * part_size: (i + 1) * part_size] for i in range(process_count)]

In [6]:
%%timeit
sum(arr)

10 loops, best of 3: 109 ms per loop


In [3]:
%%timeit
with multiprocessing.Pool(process_count) as p:
    _ = sum(p.map(sum, data))

1 loop, best of 3: 1.16 s per loop


In [24]:
def job(count):
    return sum(range(count))

In [ ]:
%%timeit
with multiprocessing.Pool(process_count) as p:
    _ = sum(p.map(job, [part_size] * process_count))

### Тестируем на вики

Асинхронный запуск поиска с помощью multiprocess

In [ ]:
class WikiReader(object):
    ...
    def run_async_multiprocess(self):
        with multiprocessing.Pool(8) as executor:
            res = list(executor.map(thread_job_multi_proc,
                                    zip([self.__tmp] * len(self.__queries), self.__queries)))
        if not all(map(lambda x: x[0][0].strip() == x[1], zip(res, self.__queries))):
            print("Failed on results order")
        return dict(res)
    ...

In [3]:
%%timeit
res = wr.run_async_multiprocess()

1 loop, best of 3: 4.37 s per loop
